In [1]:
import numpy as np
import pandas as pd
import json
import os
import glob
from itertools import combinations
import matplotlib.pyplot as plt
import sys
import cortex as cx
from pathlib import Path
from prf_expect.utils import io
from prf_expect.utils.general import extract_run_number, extract_subrun_number

In [2]:
settings = io.load_settings()

Loading settings from /Users/dionysus/Library/CloudStorage/OneDrive-Personal/Workbench/pRF_expect_pub/pRF_expect_analysis/prf_expect/settings.yml


In [3]:
# Define 
subjects = settings["general"]['subject_list']
space = settings["mri"]['space']
data_dir = os.path.join(settings["general"]["data_dir"], "data")
runs = settings["design"]['runs_per_task'] 
resample_size = np.array(settings["dm"]["resample_size"])
order = np.array([1, 2, 0])
resample_size = resample_size[order]

In [4]:
# import order of PE runs:
cwd = os.getcwd()
pe_run_order_tsv = Path(cwd).parent.parent / "run_list" / "PE_runs_order.tsv"
pe_order = pd.read_table(pe_run_order_tsv)

# change names in header to subject
rename = {}
for i, sub_nr in enumerate(pe_order):
    # fill with zeros to make the string length equal for all subjects

    rename[sub_nr] = 'sub-' + sub_nr.zfill(3)
rename['Unnamed: 0'] = 'ind'
pe_order.rename(columns=rename, inplace=True)    
display(pe_order)

,ind,sub-001,sub-002,sub-003,sub-004,sub-005,sub-006,sub-007,sub-008,sub-009,sub-010,sub-011,sub-012
0,0,"('e', 'o', 'v')","('v', 'e', 'o')","('o', 'v', 'e')","('o', 'v', 'e')","('e', 'o', 'v')","('v', 'e', 'o')","('e', 'v', 'o')","('v', 'o', 'e')","('o', 'e', 'v')","('o', 'e', 'v')","('e', 'v', 'o')","('v', 'o', 'e')"
1,1,"('o', 'v', 'e')","('o', 'v', 'e')","('v', 'e', 'o')","('e', 'o', 'v')","('v', 'e', 'o')","('e', 'o', 'v')","('o', 'e', 'v')","('o', 'e', 'v')","('v', 'o', 'e')","('e', 'v', 'o')","('v', 'o', 'e')","('e', 'v', 'o')"
2,2,"('v', 'e', 'o')","('e', 'o', 'v')","('e', 'o', 'v')","('v', 'e', 'o')","('o', 'v', 'e')","('o', 'v', 'e')","('v', 'o', 'e')","('e', 'v', 'o')","('e', 'v', 'o')","('v', 'o', 'e')","('o', 'e', 'v')","('o', 'e', 'v')"
3,3,"('e', 'v', 'o')","('v', 'o', 'e')","('o', 'e', 'v')","('o', 'e', 'v')","('e', 'v', 'o')","('v', 'o', 'e')","('e', 'o', 'v')","('v', 'e', 'o')","('o', 'v', 'e')","('o', 'v', 'e')","('e', 'o', 'v')","('v', 'e', 'o')"
4,4,"('o', 'e', 'v')","('o', 'e', 'v')","('v', 'o', 'e')","('e', 'v', 'o')","('v', 'o', 'e')","('e', 'v', 'o')","('o', 'v', 'e')","('o', 'v', 'e')","('v', 'e', 'o')","('e', 'o', 'v')","('v', 'e', 'o')","('e', 'o', 'v')"
5,5,"('v', 'o', 'e')","('e', 'v', 'o')","('e', 'v', 'o')","('v', 'o', 'e')","('o', 'e', 'v')","('o', 'e', 'v')","('v', 'e', 'o')","('e', 'o', 'v')","('e', 'o', 'v')","('v', 'e', 'o')","('o', 'v', 'e')","('o', 'v', 'e')"


### **Cut the pRF runs and save them**

In [5]:
for subject in subjects:
    # Define the input files of the runs per hemisphere
    print("Running subject: ", subject)
    runs_hemi_L = sorted(glob.glob(f'{data_dir}/derivatives/prf_data/{subject}/ses-1/psc/*task-pRF*space-{space}*_hemi-L*'))
    runs_hemi_R = sorted(glob.glob(f'{data_dir}/derivatives/prf_data/{subject}/ses-1/psc/*task-pRF*space-{space}*_hemi-R*'))
    all_runs = runs_hemi_L+runs_hemi_R

    # define and make the output directory for the subruns and averaged run files
    output_dir = f'{data_dir}/derivatives/prf_data/{subject}/ses-1/cut_and_averaged'
    os.makedirs(output_dir, exist_ok=True)

    for file in all_runs:
        
        # Load the data from each run:
    
        data = np.load(file)

        # Seperate the subruns within each run
        subrun1 = data[0:125, :]
        subrun2 = data[125:250, :]
        subrun3 = data[250:, :]
        average = np.mean(np.array([subrun1, subrun2, subrun3]), axis=0)
        
        # define the name of the files to which they are to be saved
        output_file_name = os.path.basename(file).split('.npy')[0]
        output_file_sub1 = f'{output_dir}/{output_file_name}_subrun1.npy'
        output_file_sub2 = f'{output_dir}/{output_file_name}_subrun2.npy'
        output_file_sub3 = f'{output_dir}/{output_file_name}_subrun3.npy'
        output_file_mean = f'{output_dir}/{output_file_name}_mean.npy'

        # Save the pRF subruns and average over subruns in .npy files
        np.save(output_file_sub1, subrun1)
        np.save(output_file_sub2, subrun2)
        np.save(output_file_sub3, subrun3)
        np.save(output_file_mean, average)

    # Average the three means of the three runs (for both hemispheres)
    means_L = sorted(glob.glob(f'{data_dir}/derivatives/prf_data/{subject}/ses-1/cut_and_averaged/*task-pRF*space-{space}*_hemi-L*mean*'))
    means_R = sorted(glob.glob(f'{data_dir}/derivatives/prf_data/{subject}/ses-1/cut_and_averaged/*task-pRF*space-{space}*_hemi-R*mean*'))

    for mean in [means_L, means_R]:
        mean1 = np.load(mean[0])
        mean2 = np.load(mean[1])
        mean3 = np.load(mean[2])

        total_mean = np.mean(np.array([mean1,mean2,mean3]), axis=0)
        if mean == means_L:
            mean_file_name = f'{output_dir}/{subject}_ses-1_task-pRF_space-{space}_hemi-L_desc-denoised_bold_psc_mean.npy'

        if mean == means_R:
            mean_file_name = f'{output_dir}/{subject}_ses-1_task-pRF_space-{space}_hemi-R_desc-denoised_bold_psc_mean.npy'
        
        np.save(mean_file_name, total_mean)

Running subject:  sub-001
Running subject:  sub-002
Running subject:  sub-004
Running subject:  sub-005
Running subject:  sub-007
Running subject:  sub-009
Running subject:  sub-012


## **Cut and append the task-PE files, so there are three runs:** 
- 1 with only omissions 
- 1 with the violated pRF-barpasses  
- 1 with only the sparse

In [6]:
# display pe_order again
display(pe_order)

,ind,sub-001,sub-002,sub-003,sub-004,sub-005,sub-006,sub-007,sub-008,sub-009,sub-010,sub-011,sub-012
0,0,"('e', 'o', 'v')","('v', 'e', 'o')","('o', 'v', 'e')","('o', 'v', 'e')","('e', 'o', 'v')","('v', 'e', 'o')","('e', 'v', 'o')","('v', 'o', 'e')","('o', 'e', 'v')","('o', 'e', 'v')","('e', 'v', 'o')","('v', 'o', 'e')"
1,1,"('o', 'v', 'e')","('o', 'v', 'e')","('v', 'e', 'o')","('e', 'o', 'v')","('v', 'e', 'o')","('e', 'o', 'v')","('o', 'e', 'v')","('o', 'e', 'v')","('v', 'o', 'e')","('e', 'v', 'o')","('v', 'o', 'e')","('e', 'v', 'o')"
2,2,"('v', 'e', 'o')","('e', 'o', 'v')","('e', 'o', 'v')","('v', 'e', 'o')","('o', 'v', 'e')","('o', 'v', 'e')","('v', 'o', 'e')","('e', 'v', 'o')","('e', 'v', 'o')","('v', 'o', 'e')","('o', 'e', 'v')","('o', 'e', 'v')"
3,3,"('e', 'v', 'o')","('v', 'o', 'e')","('o', 'e', 'v')","('o', 'e', 'v')","('e', 'v', 'o')","('v', 'o', 'e')","('e', 'o', 'v')","('v', 'e', 'o')","('o', 'v', 'e')","('o', 'v', 'e')","('e', 'o', 'v')","('v', 'e', 'o')"
4,4,"('o', 'e', 'v')","('o', 'e', 'v')","('v', 'o', 'e')","('e', 'v', 'o')","('v', 'o', 'e')","('e', 'v', 'o')","('o', 'v', 'e')","('o', 'v', 'e')","('v', 'e', 'o')","('e', 'o', 'v')","('v', 'e', 'o')","('e', 'o', 'v')"
5,5,"('v', 'o', 'e')","('e', 'v', 'o')","('e', 'v', 'o')","('v', 'o', 'e')","('o', 'e', 'v')","('o', 'e', 'v')","('v', 'e', 'o')","('e', 'o', 'v')","('e', 'o', 'v')","('v', 'e', 'o')","('o', 'v', 'e')","('o', 'v', 'e')"


In [7]:
# For completed subjects
for subject in subjects:
    # Define the input files of the runs per hemisphere
    PE_runs_hemi_L = sorted(glob.glob(f'{data_dir}/derivatives/prf_data/{subject}/ses-1/psc/*task-PE*space-{space}*_hemi-L*'))
    PE_runs_hemi_R = sorted(glob.glob(f'{data_dir}/derivatives/prf_data/{subject}/ses-1/psc/*task-PE*space-{space}*_hemi-R*'))
    all_runs = PE_runs_hemi_L + PE_runs_hemi_R
    
    # define and make the output directory for the subruns and averaged run files
    output_dir = f'{data_dir}/derivatives/prf_data/{subject}/ses-1/cut_and_averaged'
    os.makedirs(output_dir, exist_ok=True)

    # categorize what type of subrun is in what position of each run, then cut and append it to a new run
    # since the df stores the order of the subruns in a string, we will use the index (2, 7, 12) to get the order of the subruns
    sparse_run = np.zeros((125, 163842))
    violation_run = np.zeros((125, 163842))
    omission_run = np.zeros((125, 163842))

    for file_list in (PE_runs_hemi_L, PE_runs_hemi_R):
        for i, file in enumerate(file_list):
            data = np.load(file)
            run = extract_run_number(file)
            run_index = run - 1
    
            if run == 1:

                    # get the first sub-run
                if pe_order.loc[run_index, subject][2] == 'e':
                    sparse_run = data[0:125,:]
                    print(f'subrun 1 for {subject} in run {run} is sparse')
                elif pe_order.loc[run_index, subject][2] == 'v':
                    violation_run = data[0:125,:]
                    print(f'subrun 1 for {subject} in run {run} is a violation')
                elif pe_order.loc[run_index, subject][2] == 'o':
                    omission_run = data[0:125,:]
                    print(f'subrun 1 for {subject} in run {run} is an omission')
                    # get the first sub-run
                if pe_order.loc[run_index, subject][7] == 'e':
                    sparse_run = data[125:250,:]
                    print(f'subrun 2 for {subject} in run {run} is sparse')
                elif pe_order.loc[run_index, subject][7] == 'v':
                    violation_run = data[125:250,:]
                    print(f'subrun 2 for {subject} in run {run} is a violation')
                elif pe_order.loc[run_index, subject][7] == 'o':
                    omission_run = data[125:250,:]
                    print(f'subrun 2 for {subject} in run {run} is an omission')
                    # get the first sub-run
                if pe_order.loc[run_index, subject][12] == 'e':
                    sparse_run = data[250:,:]
                    print(f'subrun 3 for {subject} in run {run} is sparse')
                elif pe_order.loc[run_index, subject][12] == 'v':
                    violation_run = data[250:,:]
                    print(f'subrun 3 for {subject} in run {run} is a violation')
                elif pe_order.loc[run_index, subject][12] == 'o':
                    omission_run = data[250:,:]
                    print(f'subrun 3 for {subject} in run {run} is an omission')

            elif run == 2:
            
                # get the first sub-run
                if pe_order.loc[run_index, subject][2] == 'e':
                    sparse_run = data[0:125,:]
                    print(f'subrun 1 for {subject} in run {run} is sparse')
                elif pe_order.loc[run_index, subject][2] == 'v':
                    violation_run = data[0:125,:]
                    print(f'subrun 1 for {subject} in run {run} is a violation')
                elif pe_order.loc[run_index, subject][2] == 'o':
                    omission_run = data[0:125,:]
                    print(f'subrun 1 for {subject} in run {run} is an omission')
                # get the second sub-run
                if pe_order.loc[run_index, subject][7] == 'e':
                    sparse_run = data[125:250,:]
                    print(f'subrun 2 for {subject} in run {run} is sparse')
                elif pe_order.loc[run_index, subject][7] == 'v':
                    violation_run = data[125:250,:]
                    print(f'subrun 2 for {subject} in run {run} is a violation')
                elif pe_order.loc[run_index, subject][7] == 'o':
                    omission_run = data[125:250,:]
                    print(f'subrun 2 for {subject} in run {run} is an omission')
                # get the third sub-run
                if pe_order.loc[run_index, subject][12] == 'e':
                    sparse_run = data[250:,:]
                    print(f'subrun 3 for {subject} in run {run} is sparse')
                elif pe_order.loc[run_index, subject][12] == 'v':
                    violation_run = data[250:,:]
                    print(f'subrun 3 for {subject} in run {run} is a violation')
                elif pe_order.loc[run_index, subject][12] == 'o':
                    omission_run = data[250:,:]
                    print(f'subrun 3 for {subject} in run {run} is an omission')
                
            elif run == 3:

                # get the first sub-run
                if pe_order.loc[run_index, subject][2] == 'e':
                    sparse_run = data[0:125,:]
                    print(f'subrun 1 for {subject} in run {run} is sparse')
                elif pe_order.loc[run_index, subject][2] == 'v':
                    violation_run = data[0:125,:]
                    print(f'subrun 1 for {subject} in run {run} is a violation')
                elif pe_order.loc[run_index, subject][2] == 'o':
                    omission_run = data[0:125,:]
                    print(f'subrun 1 for {subject} in run {run} is an omission')
                # get the second sub-run
                if pe_order.loc[run_index, subject][7] == 'e':
                    sparse_run = data[125:250,:]
                    print(f'subrun 2 for {subject} in run {run} is sparse')
                elif pe_order.loc[run_index, subject][7] == 'v':
                    violation_run = data[125:250,:]
                    print(f'subrun 2 for {subject} in run {run} is a violation')
                elif pe_order.loc[run_index, subject][7] == 'o':
                    omission_run = data[125:250,:]
                    print(f'subrun 2 for {subject} in run {run} is an omission')
                # get the third sub-run
                if pe_order.loc[run_index, subject][12] == 'e':
                    sparse_run = data[250:,:]
                    print(f'subrun 3 for {subject} in run {run} is sparse')
                elif pe_order.loc[run_index, subject][12] == 'v':
                    violation_run = data[250:,:]
                    print(f'subrun 3 for {subject} in run {run} is a violation')
                elif pe_order.loc[run_index, subject][12] == 'o':
                    omission_run = data[250:,:]
                    print(f'subrun 3 for {subject} in run {run} is an omission')

            new_file_name = os.path.basename(file).split('.npy')[0]
            print(new_file_name)
            # new_file_name = re.sub(r'_run-\d{2}', r'_run-\d{2}', new_file_name)
            output_fname_sparse = os.path.join(output_dir, new_file_name + '_sparse.npy')
            output_fname_violation = os.path.join(output_dir, new_file_name + '_violation.npy')
            output_fname_omission = os.path.join(output_dir, new_file_name + '_omission.npy')

            print(f'Saving the run with shape {np.shape(sparse_run)} to: {output_fname_sparse}')
            print(f'Saving the run with shape {np.shape(violation_run)} to: {output_fname_violation}')
            print(f'Saving the run with shape {np.shape(omission_run)} to: {output_fname_omission}')

            np.save(output_fname_sparse, sparse_run)
            np.save(output_fname_violation, violation_run)
            np.save(output_fname_omission, omission_run)


subrun 1 for sub-001 in run 1 is sparse
subrun 2 for sub-001 in run 1 is an omission
subrun 3 for sub-001 in run 1 is a violation
sub-001_ses-1_task-PE_run-01_space-fsaverage_hemi-L_desc-denoised_bold_psc
Saving the run with shape (125, 163842) to: /Users/dionysus/Library/CloudStorage/OneDrive-Personal/Workbench/pRF_expect_pub/data/derivatives/prf_data/sub-001/ses-1/cut_and_averaged/sub-001_ses-1_task-PE_run-01_space-fsaverage_hemi-L_desc-denoised_bold_psc_sparse.npy
Saving the run with shape (125, 163842) to: /Users/dionysus/Library/CloudStorage/OneDrive-Personal/Workbench/pRF_expect_pub/data/derivatives/prf_data/sub-001/ses-1/cut_and_averaged/sub-001_ses-1_task-PE_run-01_space-fsaverage_hemi-L_desc-denoised_bold_psc_violation.npy
Saving the run with shape (125, 163842) to: /Users/dionysus/Library/CloudStorage/OneDrive-Personal/Workbench/pRF_expect_pub/data/derivatives/prf_data/sub-001/ses-1/cut_and_averaged/sub-001_ses-1_task-PE_run-01_space-fsaverage_hemi-L_desc-denoised_bold_psc_om

## **Cut and append the task-PE DMs, so there are three runs:** 
- 1 with only omissions 
- 1 with the violated pRF-barpasses  
- 1 with only the sparse

In [8]:
for subject in subjects:
    # Define the input files of the runs per hemisphere
    PE_dms = sorted(glob.glob(f'{data_dir}/derivatives/prf_data/{subject}/ses-1/dms/dm_task-PE_run-0[1-3].npy'))
    
    # define and make the output directory for the subruns and averaged run files
    output_dir = f'{data_dir}/derivatives/prf_data/{subject}/ses-1/dms'
    os.makedirs(output_dir, exist_ok=True)

    # categorize what type of subrun is in what position of each run, then cut and append it to a new run

    sparse_run = np.zeros(resample_size)
    violation_run = np.zeros(resample_size)
    omission_run = np.zeros(resample_size)
    for i, file in enumerate(PE_dms):
        print(f'loading file {file}')
        data = np.load(file)
        if data.shape[-1] != 375:
            raise ValueError(f'File {file} has shape {data.shape} instead of (63,63,375)')
        run = extract_run_number(file)
        run_index = run - 1
        # for j, config in enumerate(pe_order[subject][i][:]):
            
        # # get the subruns of the first run in new variables for each configuration
        if run == 1:
        # get the first sub-run

            if pe_order.loc[run_index, subject][2] == 'e':
                sparse_run = data[:,:,:125]
                print(f'subrun 1 for {subject} in run {run} is sparse')
            elif pe_order.loc[run_index, subject][2] == 'v':
                violation_run = data[:,:,:125]
                print(f'subrun 1 for {subject} in run {run} is a violation')
            elif pe_order.loc[run_index, subject][2] == 'o':
                omission_run = data[:,:,:125]
                print(f'subrun 1 for {subject} in run {run} is an omission')
            
            if pe_order.loc[run_index, subject][7] == 'e':
                sparse_run = data[:,:,125:250]
                print(f'subrun 2 for {subject} in run {run} is sparse')
            elif pe_order.loc[run_index, subject][7] == 'v':
                violation_run = data[:,:,125:250]
                print(f'subrun 2 for {subject} in run {run} is a violation')
            elif pe_order.loc[run_index, subject][7] == 'o':
                omission_run = data[:,:,125:250]
                print(f'subrun 2 for {subject} in run {run} is an omission')
            
            if pe_order.loc[run_index, subject][12] == 'e':
                sparse_run = data[:,:,250:]
                print(f'subrun 3 for {subject} in run {run} is sparse')
            elif pe_order.loc[run_index, subject][12] == 'v':
                violation_run = data[:,:,250:]
                print(f'subrun 3 for {subject} in run {run} is a violation')
            elif pe_order.loc[run_index, subject][12] == 'o':
                omission_run = data[:,:,250:]
                print(f'subrun 3 for {subject} in run {run} is an omission')
        
        elif run == 2:

            # get the second sub-run
            if pe_order.loc[run_index, subject][2] == 'e':
                sparse_run = data[:,:,:125]
                print(f'subrun 1 for {subject} in run {run} is sparse')
            elif pe_order.loc[run_index, subject][2] == 'v':
                violation_run = data[:,:,:125]
                print(f'subrun 1 for {subject} in run {run} is a violation')
            elif pe_order.loc[run_index, subject][2] == 'o':
                omission_run = data[:,:,:125]
                print(f'subrun 1 for {subject} in run {run} is an omission')
            if pe_order.loc[run_index, subject][7] == 'e':
                sparse_run = data[:,:,125:250]
                print(f'subrun 2 for {subject} in run {run} is sparse')
            elif pe_order.loc[run_index, subject][7] == 'v':
                violation_run = data[:,:,125:250]
                print(f'subrun 2 for {subject} in run {run} is a violation')
            elif pe_order.loc[run_index, subject][7] == 'o':
                omission_run = data[:,:,125:250]
                print(f'subrun 2 for {subject} in run {run} is an omission')
            if pe_order.loc[run_index, subject][12] == 'e':
                sparse_run = data[:,:,250:]
                print(f'subrun 3 for {subject} in run {run} is sparse')
            elif pe_order.loc[run_index, subject][12] == 'v':
                violation_run = data[:,:,250:]
                print(f'subrun 3 for {subject} in run {run} is a violation')
            elif pe_order.loc[run_index, subject][12] == 'o':
                omission_run = data[:,:,250:]
                print(f'subrun 3 for {subject} in run {run} is an omission')

        elif run == 3:

            # get the third sub-run
            if pe_order.loc[run_index, subject][2] == 'e':
                sparse_run = data[:,:,:125]
                print(f'subrun 1 for {subject} in run {run} is sparse')
            elif pe_order.loc[run_index, subject][2] == 'v':
                violation_run = data[:,:,:125]
                print(f'subrun 1 for {subject} in run {run} is a violation')
            elif pe_order.loc[run_index, subject][2] == 'o':
                omission_run = data[:,:,:125]
                print(f'subrun 1 for {subject} in run {run} is an omission')
            if pe_order.loc[run_index, subject][7] == 'e':
                sparse_run = data[:,:,125:250]
                print(f'subrun 2 for {subject} in run {run} is sparse')
            elif pe_order.loc[run_index, subject][7] == 'v':
                violation_run = data[:,:,125:250]
                print(f'subrun 2 for {subject} in run {run} is a violation')
            elif pe_order.loc[run_index, subject][7] == 'o':
                omission_run = data[:,:,125:250]
                print(f'subrun 2 for {subject} in run {run} is an omission')
            if pe_order.loc[run_index, subject][12] == 'e':
                sparse_run = data[:,:,250:]
                print(f'subrun 3 for {subject} in run {run} is sparse')
            elif pe_order.loc[run_index, subject][12] == 'v':
                violation_run = data[:,:,250:]
                print(f'subrun 3 for {subject} in run {run} is a violation')
            elif pe_order.loc[run_index, subject][12] == 'o':
                omission_run = data[:,:,250:]
                print(f'subrun 3 for {subject} in run {run} is an omission')

        new_file_name = os.path.basename(file).split('.npy')[0]
        # new_file_name = re.sub(r'_run-\d{2}', '', new_file_name)
        output_fname_sparse = output_dir + f'/{subject}_ses-1_task-sparse_{runs[run_index]}_dm.npy'
        output_fname_violation = output_dir + f'/{subject}_ses-1_task-violation_{runs[run_index]}_dm.npy'
        output_fname_omission = output_dir + f'/{subject}_ses-1_task-omission_{runs[run_index]}_dm.npy'

        print(f'Saving the run with shape {np.shape(sparse_run)} to: {output_fname_sparse}')
        print(f'Saving the run with shape {np.shape(violation_run)} to: {output_fname_violation}')
        print(f'Saving the run with shape {np.shape(omission_run)} to: {output_fname_omission}')

        np.save(output_fname_sparse, sparse_run)
        np.save(output_fname_violation, violation_run)
        np.save(output_fname_omission, omission_run)


loading file /Users/dionysus/Library/CloudStorage/OneDrive-Personal/Workbench/pRF_expect_pub/data/derivatives/prf_data/sub-001/ses-1/dms/dm_task-PE_run-01.npy
subrun 1 for sub-001 in run 1 is sparse
subrun 2 for sub-001 in run 1 is an omission
subrun 3 for sub-001 in run 1 is a violation
Saving the run with shape (63, 63, 125) to: /Users/dionysus/Library/CloudStorage/OneDrive-Personal/Workbench/pRF_expect_pub/data/derivatives/prf_data/sub-001/ses-1/dms/sub-001_ses-1_task-sparse_run-01_dm.npy
Saving the run with shape (63, 63, 125) to: /Users/dionysus/Library/CloudStorage/OneDrive-Personal/Workbench/pRF_expect_pub/data/derivatives/prf_data/sub-001/ses-1/dms/sub-001_ses-1_task-violation_run-01_dm.npy
Saving the run with shape (63, 63, 125) to: /Users/dionysus/Library/CloudStorage/OneDrive-Personal/Workbench/pRF_expect_pub/data/derivatives/prf_data/sub-001/ses-1/dms/sub-001_ses-1_task-omission_run-01_dm.npy
loading file /Users/dionysus/Library/CloudStorage/OneDrive-Personal/Workbench/pRF_